## Speech-to-Text Conversion Using ESPnet Pre-Trained Model


In [ ]:
#Installing the python package to containing the pre trained ESPnet models
! pip install -q espnet_model_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
#-------Downloading and upacking espnet model -----


import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


lang = 'en'
fs = 16000    #sampling rate
tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave'
d = ModelDownloader()
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.4,
    beam_size=15,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.lower()
    return text.translate(str.maketrans('', '', string.punctuation))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
https://zenodo.org/record/4585558/files/asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave.zip?download=1: 100%|██████████| 772M/772M [00:32<00:00, 25.0MB/s]
/usr/local/lib/python3.10/dist-packages/espnet_model_zoo/downloader.py:364: UserWarning: Not validating checksum
  warnings.warn("Not validating checksum")


In [ ]:
#------Extracting all files of audio from the google drive--------


import os
import glob


base_dir = '/content/drive/MyDrive/test-clean'
print(base_dir)
filepath=[]

for root, dirs, files in os.walk(base_dir):
    for file in files:
      if file.endswith(".flac")==True:
        filepathstemp=os.path.join(root,file)
        filepath.append(os.path.join(root,file))

print(len(filepath))


/content/drive/MyDrive/test-clean
2620


In [ ]:
#-----Importing Required Libraries-------
import numpy as np
import pandas as pd
import soundfile
import librosa.display

In [ ]:
#------------converting the imported audio file link to a dataframe---------------

df = pd.DataFrame({'id': filepath})
df.head()

,id
0,/content/drive/MyDrive/test-clean/7176/92135/7...
1,/content/drive/MyDrive/test-clean/7176/92135/7...
2,/content/drive/MyDrive/test-clean/7176/92135/7...
3,/content/drive/MyDrive/test-clean/7176/92135/7...
4,/content/drive/MyDrive/test-clean/7176/92135/7...


In [ ]:
#----------Reading meta data file containing target text-------------------


headers=['id','text_output','value','sr']
df2 = pd.read_csv('/content/drive/MyDrive/metadata_test_clean.tsv', sep='\t', header=None, names=headers)
df2.head()

,id,text_output,value,sr
0,originals/test-clean/4970/29095/4970-29095-000...,she was tired of other things,2.865,16000
1,originals/test-clean/4970/29095/4970-29095-000...,she tried this morning an air or two upon the ...,13.910,16000
2,originals/test-clean/4970/29095/4970-29095-000...,well mother said the young student looking up ...,5.480,16000
3,originals/test-clean/4970/29095/4970-29095-000...,i hope thee told the elders that father and i ...,10.020,16000
4,originals/test-clean/4970/29095/4970-29095-000...,i heard father tell cousin abner that he was w...,9.610,16000


In [ ]:
#----modifying the id column of metadata so as to make it compatible with the previous dataframe to merge the two dataframes------------
df2['id']=df2['id'].str.replace('originals/','/content/drive/MyDrive/')

df_new = pd.merge(df,df2, left_on='id', right_on='id', how='left')
df_new.tail()

,id,text_output,value,sr
2615,/content/drive/MyDrive/test-clean/2300/131720/...,i think he was perhaps more appreciative than ...,11.060,16000
2616,/content/drive/MyDrive/test-clean/2300/131720/...,for some years it was not found feasible to op...,9.605,16000
2617,/content/drive/MyDrive/test-clean/2300/131720/...,the strong position held by the edison system ...,14.820,16000
2618,/content/drive/MyDrive/test-clean/2300/131720/...,mister edison was a leader far ahead of the time,3.750,16000
2619,/content/drive/MyDrive/test-clean/2300/131720/...,why if we erect a station at the falls it is a...,6.900,16000


In [ ]:
#---Data preprocessing : checking for any missing or duplicated values in the datframe-----

print(df_new.duplicated().sum())
print(df_new.isnull().sum())
df_new.info()


0
id             0
text_output    0
value          0
sr             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2620 entries, 0 to 2619
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2620 non-null   object 
 1   text_output  2620 non-null   object 
 2   value        2620 non-null   float64
 3   sr           2620 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 102.3+ KB


In [ ]:
import numpy as np
from itertools import islice

In [ ]:
#----Getting ouptput text from the model--------


output_file_path = '/content/model_asr.txt'
files_path = df_new['id']
with open(output_file_path, "w") as output_file:
  for files in df_new['id']:
    # start_time=time.time()
    speech, rate = soundfile.read(files)
    nbests = speech2text(speech)
    text, *_= nbests[0]
    # print(f'{text_normalizer(text)}')
    output_file.write(f'{text_normalizer(text)}\n')
    # print(time.time()-start_time)

In [ ]:
#---installing torchmetrics------
!pip install torchmetrics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 7.6 MB/s eta 0:00:00


#NOTE :::  The output of model is stored in text file which is converted to a csv file which is then imported in next cell.

In [ ]:
#----reading the output data stored in the model_asr_output csv file----------
columns=['preds']
predictions = pd.read_csv('/content/model_asr_output.csv', header=None, names=columns)
predictions.head()


,preds
0,the crowd drift off leaping the hero and terra...
1,it is to let hemlet if that happened to be the...
2,and so im until you get to the end im going t...
3,entered lloyd arthur fleetnet
4,how you may be wondering are you going to begi...


In [ ]:
#-------making the dataframe such that it contains target and output text--
df_new.drop(columns=['sr','value','id'], axis=1, inplace=True)
df_new['preds']=predictions['preds']
df_new.head()

,text_output,preds
0,the crowd drifts off leaving the hero and hero...,the crowd drift off leaping the hero and terra...
1,it is to let hamlet if that happen to be the n...,it is to let hemlet if that happened to be the...
2,and so on till you get to the end when ophelia...,and so im until you get to the end im going t...
3,enter lord arthur fluffinose,entered lloyd arthur fleetnet
4,how you may be wondering are you to begin your...,how you may be wondering are you going to begi...


In [ ]:
#---Evaluating CER and WER--------

from torchmetrics.text import WordErrorRate,CharErrorRate
wer = WordErrorRate()
reference_text = df_new['preds'].astype(str)
target_text = df_new['text_output'].astype(str)
wer_score = wer(reference_text, target_text)
cer = CharErrorRate()
cer_score = cer(reference_text, target_text)


In [ ]:
#----printing cer and wer----------

print(f'The Word Error Rate is: {round(wer_score.item(),3)}')
print(f'The Character Error Rate : {round(cer_score.item(),3)}')

The Word Error Rate is: 0.264
The Character Error Rate : 0.153
